<a href="https://colab.research.google.com/github/rtajeong/M2/blob/main/gg_18_%ED%86%B5%EA%B3%84%EC%A0%81%EB%B6%84%EC%84%9D_%EC%A3%BC%ED%83%9D%EA%B0%80%EA%B2%A9%EC%98%88%EC%B8%A1_rev1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## 통계적 분석

- 상관관계가 큰 변수 찾기
- 결측치 처리
- 이상치 처리
- 로그 변환

### 데이터 (주택가격 예측 데이터)
- 아래 캐글 데이터를 다운받아 'data/house_data'폴더에 저장한다.
  https://www.kaggle.com/c/house-prices-advanced-regression-techniques/data

- https://medium.com/mighty-data-science-bootcamp/eda-%EB%8D%B0%EC%9D%B4%ED%84%B0-%EC%84%A4%EB%AA%85%EC%84%9C%EC%97%90%EC%84%9C-%EC%8B%9C%EC%9E%91%ED%95%98%EA%B8%B0-230060b9fc17

In [4]:
!pip install seaborn

In [5]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
from scipy.stats import norm
from sklearn.preprocessing import StandardScaler
from scipy import stats
import warnings
warnings.filterwarnings('ignore')
%matplotlib inline

In [7]:
df_train = pd.read_csv('house_train.csv')
df_train.head()

,Id,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,LotConfig,LandSlope,Neighborhood,Condition1,Condition2,BldgType,HouseStyle,OverallQual,OverallCond,YearBuilt,YearRemodAdd,RoofStyle,RoofMatl,Exterior1st,Exterior2nd,MasVnrType,MasVnrArea,ExterQual,ExterCond,Foundation,BsmtQual,BsmtCond,BsmtExposure,BsmtFinType1,BsmtFinSF1,BsmtFinType2,BsmtFinSF2,BsmtUnfSF,TotalBsmtSF,Heating,...,CentralAir,Electrical,1stFlrSF,2ndFlrSF,LowQualFinSF,GrLivArea,BsmtFullBath,BsmtHalfBath,FullBath,HalfBath,BedroomAbvGr,KitchenAbvGr,KitchenQual,TotRmsAbvGrd,Functional,Fireplaces,FireplaceQu,GarageType,GarageYrBlt,GarageFinish,GarageCars,GarageArea,GarageQual,GarageCond,PavedDrive,WoodDeckSF,OpenPorchSF,EnclosedPorch,3SsnPorch,ScreenPorch,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition,SalePrice
0,1,60,RL,65.0,8450,Pave,NaN,Reg,Lvl,AllPub,Inside,Gtl,CollgCr,Norm,Norm,1Fam,2Story,7,5,2003,2003,Gable,CompShg,VinylSd,VinylSd,BrkFace,196.0,Gd,TA,PConc,Gd,TA,No,GLQ,706,Unf,0,150,856,GasA,...,Y,SBrkr,856,854,0,1710,1,0,2,1,3,1,Gd,8,Typ,0,NaN,Attchd,2003.0,RFn,2,548,TA,TA,Y,0,61,0,0,0,0,NaN,NaN,NaN,0,2,2008,WD,Normal,208500
1,2,20,RL,80.0,9600,Pave,NaN,Reg,Lvl,AllPub,FR2,Gtl,Veenker,Feedr,Norm,1Fam,1Story,6,8,1976,1976,Gable,CompShg,MetalSd,MetalSd,None,0.0,TA,TA,CBlock,Gd,TA,Gd,ALQ,978,Unf,0,284,1262,GasA,...,Y,SBrkr,1262,0,0,1262,0,1,2,0,3,1,TA,6,Typ,1,TA,Attchd,1976.0,RFn,2,460,TA,TA,Y,298,0,0,0,0,0,NaN,NaN,NaN,0,5,2007,WD,Normal,181500
2,3,60,RL,68.0,11250,Pave,NaN,IR1,Lvl,AllPub,Inside,Gtl,CollgCr,Norm,Norm,1Fam,2Story,7,5,2001,2002,Gable,CompShg,VinylSd,VinylSd,BrkFace,162.0,Gd,TA,PConc,Gd,TA,Mn,GLQ,486,Unf,0,434,920,GasA,...,Y,SBrkr,920,866,0,1786,1,0,2,1,3,1,Gd,6,Typ,1,TA,Attchd,2001.0,RFn,2,608,TA,TA,Y,0,42,0,0,0,0,NaN,NaN,NaN,0,9,2008,WD,Normal,223500
3,4,70,RL,60.0,9550,Pave,NaN,IR1,Lvl,AllPub,Corner,Gtl,Crawfor,Norm,Norm,1Fam,2Story,7,5,1915,1970,Gable,CompShg,Wd Sdng,Wd Shng,None,0.0,TA,TA,BrkTil,TA,Gd,No,ALQ,216,Unf,0,540,756,GasA,...,Y,SBrkr,961,756,0,1717,1,0,1,0,3,1,Gd,7,Typ,1,Gd,Detchd,1998.0,Unf,3,642,TA,TA,Y,0,35,272,0,0,0,NaN,NaN,NaN,0,2,2006,WD,Abnorml,140000
4,5,60,RL,84.0,14260,Pave,NaN,IR1,Lvl,AllPub,FR2,Gtl,NoRidge,Norm,Norm,1Fam,2Story,8,5,2000,2000,Gable,CompShg,VinylSd,VinylSd,BrkFace,350.0,Gd,TA,PConc,Gd,TA,Av,GLQ,655,Unf,0,490,1145,GasA,...,Y,SBrkr,1145,1053,0,2198,1,0,2,1,4,1,Gd,9,Typ,1,TA,Attchd,2000.0,RFn,3,836,TA,TA,Y,192,84,0,0,0,0,NaN,NaN,NaN,0,12,2008,WD,Normal,250000


In [ ]:
df_train.columns

### 특성값 설명
- SalePrice - the property's sale price in dollars (target variable)
- MSSubClass: The building class
- MSZoning: The general zoning classification
- LotFrontage: Linear feet of street connected to property
- LotArea: Lot size in square feet
- Street: Type of road access
- Alley: Type of alley access
- LotShape: General shape of property
- LandContour: Flatness of the property
- Utilities: Type of utilities available
- LotConfig: Lot configuration
- LandSlope: Slope of property
- Neighborhood: Physical locations within Ames city limits
- Condition1: Proximity to main road or railroad
- Condition2: Proximity to main road or railroad (if a second is present)
- BldgType: Type of dwelling
- HouseStyle: Style of dwelling
- OverallQual: Overall material and finish quality
- OverallCond: Overall condition rating
- YearBuilt: Original construction date
- YearRemodAdd: Remodel date
- RoofStyle: Type of roof
- RoofMatl: Roof material
- Exterior1st: Exterior covering on house
- Exterior2nd: Exterior covering on house (if more than one material)
- MasVnrType: Masonry veneer type
- MasVnrArea: Masonry veneer area in square feet
- ExterQual: Exterior material quality
- ExterCond: Present condition of the material on the exterior
- Foundation: Type of foundation
- BsmtQual: Height of the basement
- BsmtCond: General condition of the basement
- BsmtExposure: Walkout or garden level basement walls
- BsmtFinType1: Quality of basement finished area
- BsmtFinSF1: Type 1 finished square feet
- BsmtFinType2: Quality of second finished area (if present)
- BsmtFinSF2: Type 2 finished square feet
- BsmtUnfSF: Unfinished square feet of basement area
- TotalBsmtSF: Total square feet of basement area
- Heating: Type of heating
- HeatingQC: Heating quality and condition
- CentralAir: Central air conditioning
- Electrical: Electrical system
- 1stFlrSF: First Floor square feet
- 2ndFlrSF: Second floor square feet
- LowQualFinSF: Low quality finished square feet (all floors)
- GrLivArea: Above grade (ground) living area square feet
- BsmtFullBath: Basement full bathrooms
- BsmtHalfBath: Basement half bathrooms
- FullBath: Full bathrooms above grade
- HalfBath: Half baths above grade
- Bedroom: Number of bedrooms above basement level
- Kitchen: Number of kitchens
- KitchenQual: Kitchen quality
- TotRmsAbvGrd: Total rooms above grade (does not include bathrooms)
- Functional: Home functionality rating
- Fireplaces: Number of fireplaces
- FireplaceQu: Fireplace quality
- GarageType: Garage location
- GarageYrBlt: Year garage was built
- GarageFinish: Interior finish of the garage
- GarageCars: Size of garage in car capacity
- GarageArea: Size of garage in square feet
- GarageQual: Garage quality
- GarageCond: Garage condition
- PavedDrive: Paved driveway
- WoodDeckSF: Wood deck area in square feet
- OpenPorchSF: Open porch area in square feet
- EnclosedPorch: Enclosed porch area in square feet
- 3SsnPorch: Three season porch area in square feet
- ScreenPorch: Screen porch area in square feet
- PoolArea: Pool area in square feet
- PoolQC: Pool quality
- Fence: Fence quality
- MiscFeature: Miscellaneous feature not covered in other categories
- MiscVal: $Value of miscellaneous feature
- MoSold: Month Sold
- YrSold: Year Sold
- SaleType: Type of sale
- SaleCondition: Condition of sale

In [ ]:
df_train.shape

### Target (목적변수) 의 특성 파악

In [ ]:
# 목적 변수의 통계적 특성
df_train['SalePrice'].describe()

In [ ]:
# 히스토그램
plt.hist(df_train['SalePrice'], bins=50)

In [ ]:
sns.kdeplot(df_train['SalePrice'])

In [ ]:
# 히스토그램
# sns.distplot(): This function combines the matplotlib hist function (with automatic 
# calculation of a good default bin size) with the seaborn kdeplot() and rugplot() 
# functions. It can also fit scipy.stats distributions and plot the estimated PDF 
# over the data.
# Flexibly plot a univariate distribution of observations. 
# y-axis is density rather than  count.
sns.distplot(df_train['SalePrice'], kde=True); # kde - kernel density estimation

#### 다음과 같은 특성을 발견할 수 있다.

- Deviate from the normal distribution
- Have appreciable positive skewness
- Show peakedness

- 첨도: 뾰족한 정도가 표준정규뷴포와 같으며, 0 보다 크면 그보다 더 뾰족함.
  (관측치들이 어느 정도 집중적으로 중심에 몰려 있는가를 측정할 때 사용)
  첨도값(K)이 3에 가까우면 산포도가 정규분포에 가깝다. 3보다 작을 경우에는(K<3) 
  정규분포보다 더 완만하게 납작한 분포로 판단할 수 있으며, 첨도값이 3보다 큰 
  양수이면(K>3) 산포는 정규분포보다 더 뾰족한 분포로 생각할 수 있다

In [ ]:
#skewness and kurtosis (kurto'sis)
print("Skewness: %f" % df_train['SalePrice'].skew())
print("Kurtosis: %f" % df_train['SalePrice'].kurt())          # old-type
print("Kurtosis: {:f}".format(df_train['SalePrice'].kurt()))  # string format()

In [ ]:
sns.distplot(np.log(df_train['SalePrice']), kde=True)  # log-normal dist

### 어떤 변수들이 목적변수와 연관성이 높은가? 
- 변수들간의 상관 관계는 correlation (상관관계) 을 보면 알 수 있다. 
- 상관관계를 보기 전에 먼저 target 과 특정 변수와의 관계를 보고 싶다면...

In [ ]:
# 산포도를 통해 grlivarea 와 saleprice 의 관계를 보자
var = 'GrLivArea'
data = pd.concat([df_train['SalePrice'], df_train[var]], axis=1)
data.head(2)

In [ ]:
data.plot.scatter(x=var, y='SalePrice', ylim=(0,800000));

In [ ]:
# plt.scatter 로도 그릴 수 있다.
# data[var].shape, data.SalePrice.shape
# plt.scatter(data[var], data.SalePrice);

In [ ]:
# scatter plot for totalbsmtsf/saleprice
var = 'TotalBsmtSF'
data = pd.concat([df_train['SalePrice'], df_train[var]], axis=1)
data.plot.scatter(x=var, y='SalePrice', ylim=(0,800000));

### 카테고리 변수와의 관계: 박스플롯으로 보면 편리

In [ ]:
print(df_train.shape)
print(df_train['OverallQual'].unique())
print(set(df_train['OverallQual']))
print(df_train['OverallQual'].dtypes)
df_train['OverallQual'].value_counts()

In [ ]:
print(df_train['GarageCars'].unique())
df_train['GarageCars'].dtypes
df_train['GarageCars'].value_counts()

In [ ]:
# 박스플롯 overallqual/saleprice
var = 'OverallQual'
data = pd.concat([df_train['SalePrice'], df_train[var]], axis=1) 
data.head(2)

In [ ]:
data.plot.scatter(x=var, y='SalePrice', ylim=(0,800000));

### scatter plot 보다는 boxplot 으로 보는 게 좋을 듯...
- df.polt.box() 도 있지만 받아들이는 데이터의 형태가 다름 (dataframe manual 참고)

In [ ]:
# f, ax = plt.subplots(figsize=(8, 6))
plt.subplots(figsize=(8, 6))
fig = sns.boxplot(x=var, y="SalePrice", data=data)
fig.axis(ymin=0, ymax=800000);

In [ ]:
var = 'GarageCars'
data = pd.concat([df_train['SalePrice'], df_train[var]], axis=1) # axis=0 은 위아래로 추가.
data.head()
plt.subplots(figsize=(8, 6))
fig = sns.boxplot(x=var, y="SalePrice", data=data)
fig.axis(ymin=0, ymax=800000);

In [ ]:
var = 'YearBuilt'
data = pd.concat([df_train['SalePrice'], df_train[var]], axis=1)
f, ax = plt.subplots(figsize=(16, 8))
fig = sns.boxplot(x=var, y="SalePrice", data=data)
fig.axis(ymin=0, ymax=800000);
plt.xticks(rotation=90);

약한 관련성이 있으며 최근의 주택 가격이 높다. 인플레이션이 반영된 가격일 수도 있다.
유용한 특성을 선택하는 것이 문제이다. 이러한 특성 선택을 임의로 주관적으로 수행하는 것은 한계가 있다. 객관적인 방법이 필요하다.

### 객관적으로 판단하는 방법으로는...
- Correlation matrix (heatmap style).
- 'SalePrice' correlation matrix (zoomed heatmap style).
- Scatter plots between the most correlated variables

In [ ]:
## correlation matrix
# print(df_train.corr())
corrmat = df_train.corr()
# corrmat

f, ax = plt.subplots(figsize=(14, 10))
sns.heatmap(corrmat, vmax=.8, square=True);

히트맵 타입의 상관관계 지도는 유용하다. 그림을 보면 'TotalBsmtSF', '1stFlrSF', 'GarageX' 등의 상관관계가 큰 것을 알 수 있다. 경우에 따라서는 두개의 특성이 거의 동일한 정보를 포함하는 것도 있다.

가장 중요한 것은 'SalePrice'와의 상관관계이다. 이를 보면 'GrLivArea', 'TotalBsmtSF', 'OverallQual'가 관련이 많은 것을 알 수 있다. 다른 특성들도 일부 관련이 있으므로 무시할 수 없다.

#### 'SalePrice' correlation matrix (zoomed heatmap style)

In [ ]:
corrmat

In [ ]:
corrmat.sort_values(by='SalePrice', ascending=False).T.tail()   # just for checking

In [ ]:
# saleprice correlation matrix
# # df.nlargest(): Get the rows of a DataFrame sorted by the n largest values of columns.
k = 5      #number of variables for heatmap
cols = corrmat.nlargest(k, 'SalePrice')
cols

In [ ]:
cols = cols['SalePrice']; cols

In [ ]:
cols = cols.index; cols

In [ ]:
df_train[cols].values

In [ ]:
df_train[cols].corr()   # dataframe

In [ ]:
cm = df_train[cols].corr().values

In [ ]:
# same as the above (numpy 에도 동일 기능 함수가 있음)
cm2 = np.corrcoef(df_train[cols].values.T) # Return correlation coefficients.
                                          # Each row of x represents a variable, and 
                                          # each column a single observation of variables
cm2

In [ ]:
sns.set(font_scale=1.25)
hm = sns.heatmap(cm, cbar=True, annot=True, square=True,
        fmt='.2f', annot_kws={'size': 10}, 
        yticklabels=cols.values, xticklabels=cols.values)
plt.show()

상관관계가 높은 순으로 정렬했다.

* 'OverallQual', 'GrLivArea' and 'TotalBsmtSF'은 'SalePrice'와 상관관계가 크다.
* 'GarageCars' and 'GarageArea' 특성은 같은 의미를 갖는 것으로 보인다 (상관관계 0.88).
* 'TotalBsmtSF' and '1stFloor' 도 상관관계가 크다. (0.82)
* 'FullBath'?? Really? 
* 'TotRmsAbvGrd' and 'GrLivArea' 도 상관관계가 크다(0.83).

In [ ]:
# 산포도
# sns.set()
cols = ['SalePrice', 'OverallQual', 'GrLivArea', 
  'GarageCars', 'TotalBsmtSF', 'FullBath', 'YearBuilt']
sns.pairplot(df_train[cols], size = 2.5)
plt.show();

'TotalBsmtSF' and 'GrLiveArea'이 의미 있게 보인다. 산포도를 보면 직선의 한 쪽에 집중해서 나타나는 것을 알 수 있다. 지하실의 크기는 1층의 면적과 거의 같거나 작다.

'YearBuilt'도 약한 지수함수 형태를 보인다. 가격이 계속 오르고 있다.

# 결측치 처리

결측 패턴이 산발적인가, 규칙적인가?

In [ ]:
#### 참고
# df_train.isnull()
# df_train.isnull().sum()
# df_train.count()          # number of non-null elements
# df_train.isnull().sum().sort_values(ascending=False)
# df_train.isna().count(), len(df_train)
a = np.array([6,9,12])
b= np.array([3,3,3])
a/b, a/3

### 연습

In [ ]:
df_train.isna().sum()

In [ ]:
df_train.count()  # Count non-NA cells for each column or row

In [ ]:
df_train.count().head()

# original code

In [ ]:
# missing data (아래 코드는 결측치 통계를 찾는 코드이다.)
total = df_train.isnull().sum().sort_values(ascending=False)
total.head(10)

# percent = (df_train.isnull().sum()/df_train.isnull().count()).sort_values(ascending=False)
percent = (df_train.isnull().sum()/df_train.shape[0]).sort_values(ascending=False)
missing_data = pd.concat([total, percent], axis=1, keys=['Total', 'Percent'])
missing_data.head(20)

### 분석 결과
- 결측률이 15% 이상이면 이 특성은 제외하고 분석하는 것이 타당하다.('PoolQC', 'MiscFeature', 'Alley' 등) 이러한 데이터의 중요도도 생각해봐야 한다. 

- 'Garage*X*'의 결측치 비율이 같고, 중복성이 있다.'Bsmt*X*' 특성도 마찬가지

- 'MasVnrArea' and 'MasVnrType'는'YearBuilt'나 'OverallQual' 와 상관과계가 크므로 중복성이 있어 제외 가능하다.

- 'Electrical'는 한 값만 결측이므로 해당 샘플만 제거한다.

- 'Electrical' 외의 결측치를 관찰한 결과 모두 특성에서 제외하기로 한다. 

In [ ]:
df_train_org = df_train.copy()    # 만일을 위해서 (나중에 비교해 볼 필요가 있을때)

In [ ]:
df_train = df_train.drop((missing_data[missing_data['Total'] > 1]).index,1)
df_train = df_train.drop(df_train.loc[df_train['Electrical'].isnull()].index)

df_train.isnull().sum().max()            # nul 합의 최고가 0 인지 확인

In [ ]:
df_train.shape    # 18 개 columns , 1 row 제거

# 특이값(outlier)

중요한 의미나 정보를 포함할 수 있다. 표준편차나 산포도에서 찾을 수 있다.

표준 정규화를 하여 비교한다. 상위 영역에 7 이상의 표준편차 차이가 있는 것을 알 수 있다.

In [ ]:
# saleprice_scaled = StandardScaler().fit_transform(df_train['SalePrice'][:,np.newaxis])
# saleprice_scaled

saleprice_scaled = StandardScaler().fit_transform(np.array(df_train['SalePrice']).reshape(-1,1))
saleprice_scaled

In [ ]:
saleprice_scaled[:,0]

In [ ]:
low_range = saleprice_scaled[saleprice_scaled[:,0].argsort()][:10]
high_range= saleprice_scaled[saleprice_scaled[:,0].argsort()][-10:]

# argsort() : sorting 후 순서가 index 로 나타남
# >>> x = np.array([3, 1, 2])
# >>> np.argsort(x)
# array([1, 2, 0])

print('outer range (low) of the distribution:')
print(low_range)
print('\nouter range (high) of the distribution:')
print(high_range)

In [ ]:
## exercise: for newaxis - to increase the dimension
print(df_train['SalePrice'].shape)
print(type(df_train['SalePrice']))     # series
df_train['SalePrice'].head()

In [ ]:
plt.plot(df_train['SalePrice'])   # if not shown, x-axis is default len(y)

In [ ]:
plt.plot(saleprice_scaled)

In [ ]:
# for your reference
ser = df_train['SalePrice']
ser_arr = np.array(df_train['SalePrice'])
ser_arr_resh = ser_arr.reshape(-1,1)
ser_newax = ser[:, np.newaxis]

print(type(ser), type(ser_arr), type(ser_arr_resh), type(ser_newax))
print(ser.shape, ser_arr.shape, ser_arr_resh.shape, ser_newax.shape)
print('****')
print("array = ", '\n', ser_arr)
print("reshaped array = ", '\n', ser_arr_resh)
print("newaxised array = ", '\n', ser_newax )
###
print(ser.shape, ser_arr_resh.shape, ser_newax.shape)
print('----')
print(type(df_train['SalePrice'][:,np.newaxis]))    # array
print(df_train['SalePrice'][:,np.newaxis])
print(df_train['SalePrice'][:,np.newaxis].shape)
df_train['SalePrice'].values.reshape(-1,1).shape

In [ ]:
# exercise: newaxis (축을 새로 추가하기)
import numpy as np
x = np.array([1,2,3,4])
print(x, x.shape)

In [ ]:
x[:,np.newaxis]

In [ ]:
x.reshape(4,1)  # 동일한 효과

In [ ]:
x[np.newaxis,:]

In [ ]:
x.reshape(1,4)

In [ ]:
print(x[:,np.newaxis], x[:,np.newaxis].shape)
print(x[np.newaxis,:], x[np.newaxis,:].shape)

In [ ]:
print(x.reshape(-1,1), x.reshape(-1,1).shape)
print(x.reshape(1,-1), x.reshape(1,-1).shape)

### Bivariate 분석

In [ ]:
# bivariate analysis saleprice/grlivarea
var = 'GrLivArea'
data = pd.concat([df_train['SalePrice'], df_train[var]], axis=1)

data.plot.scatter(x=var, y='SalePrice', ylim=(0,800000));

우측의 두 점은 트렌드를 따르지 않는 값으로 여기서는 삭제한다. 상위의 두 점은 7~ 값을 갖는 케이스인데 트렌드를 따르므로 삭제하지 않는다.

In [ ]:
#deleting points
df_train.sort_values(by = 'GrLivArea', ascending = False)[:2]

In [ ]:
df_train = df_train.drop(df_train[df_train['Id'] == 1299].index)
df_train = df_train.drop(df_train[df_train['Id'] == 524].index)

In [ ]:
#bivariate analysis saleprice/grlivarea
var = 'TotalBsmtSF'
data = pd.concat([df_train['SalePrice'], df_train[var]], axis=1)
data.plot.scatter(x=var, y='SalePrice', ylim=(0,800000));

우측의 이상치가 있어 보이지만 그대로 둔다.

In [ ]:
#bivariate analysis saleprice/overallqual
var = 'OverallQual'
data = pd.concat([df_train['SalePrice'], df_train[var]], axis=1)
data.plot.scatter(x=var, y='SalePrice', ylim=(0,800000));

# 다음 단계

점검할 사항:

- 정규성
- 등분산성
- 선형관계성
- 상호 연관된 오류의 부재

### In the search for normality

- 히스토그램: Kurtosis and skewness (첨도 및 왜도) 점검
- Normal probability plot

In [ ]:
# 히스토그램과 정규분포 플롯
sns.distplot(df_train['SalePrice']);  #  plot the estimated PDF over the data.
# sns.distplot(df_train['SalePrice'], fit=norm);
fig = plt.figure()
# res = stats.probplot(df_train['SalePrice'], plot=plt)
# stats.probplot(): Generates a probability plot of sample data against the quantiles 
# of a specified theoretical distribution (the normal distribution by default). 
# probplot optionally calculates a best-fit line for the data and plots the results 
# using Matplotlib or a given plot function.

'SalePrice'는 정규분포는 아니다. 정점(peakedness)이 존재하고 양의 왜곡(positive skewness)를 보인다. 양의 왜곡에는 로그 변환을 하면 도움이 된다. 

In [ ]:
# 로그변환
df_train['SalePrice'] = np.log(df_train['SalePrice'])

In [ ]:
sns.distplot(df_train['SalePrice'], fit=norm);
# sns.distplot(df_train['SalePrice']);
fig = plt.figure()
# res = stats.probplot(df_train['SalePrice'], plot=plt)

'GrLivArea'에 대해서 알아보겠다.

In [ ]:
sns.distplot(df_train['GrLivArea'], fit=norm);
fig = plt.figure()
# res = stats.probplot(df_train['GrLivArea'], plot=plt)

In [ ]:
df_train['GrLivArea'] = np.log(df_train['GrLivArea'])

In [ ]:
# 로그 변환 후 플롯
sns.distplot(df_train['GrLivArea'], fit=norm);
fig = plt.figure()
# res = stats.probplot(df_train['GrLivArea'], plot=plt)

TotalBsmtSF 변수에 대한 그래프

In [ ]:
# histogram and normal probability plot
sns.distplot(df_train['TotalBsmtSF'], fit=norm);
fig = plt.figure()
# res = stats.probplot(df_train['TotalBsmtSF'], plot=plt)

In [ ]:
# pd.Series(len(df_train['TotalBsmtSF']), index=df_train.index)

값이 0인 것은 로그 변환을 할 수 없다. 0인 경우는 '지하실 없음' 등으로 표현하고 0 이상인 값에 대해서만 로그 변환을 할 수 있다.

In [ ]:
# 별도의 컬럼 추가
# area>0 이면 1, for area==0 이면 0
df_train['HasBsmt'] = pd.Series(len(df_train['TotalBsmtSF']), index=df_train.index)
df_train.head()

In [ ]:
df_train['HasBsmt'] = 0 
df_train.loc[df_train['TotalBsmtSF']>0,'HasBsmt'] = 1
df_train.head()

In [ ]:
# df_train.HasBsmt

In [ ]:
# 양수인 경우 로그 변환
df_train.loc[df_train['HasBsmt']==1,'TotalBsmtSF'] = np.log(df_train['TotalBsmtSF'])

In [ ]:
# 히스토그램
sns.distplot(df_train[df_train['TotalBsmtSF']>0]['TotalBsmtSF'], fit=norm);
fig = plt.figure()
# res = stats.probplot(df_train[df_train['TotalBsmtSF']>0]['TotalBsmtSF'], plot=plt)

### homoscedasticity (동질학)
- 산포도로 확인할 수 있다.

In [ ]:
#scatter plot
plt.scatter(df_train['GrLivArea'], df_train['SalePrice']);

In [ ]:
#scatter plot
plt.scatter(df_train[df_train['TotalBsmtSF']>0]['TotalBsmtSF'], df_train[df_train['TotalBsmtSF']>0]['SalePrice']);

# 위의 처리 이전과 이후를 비교해 보기 위해

In [ ]:
df_train_org.shape, df_train.shape

In [ ]:
###
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
df = df_train_org.copy()
corrmat = df.corr()
corrmat.sort_values(by='SalePrice', ascending=False).T.tail()

cols = corrmat.nlargest(5, 'SalePrice') 
df2 = df[cols.index]

X, y = df2.values[:,1:], df2.values[:,0]
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)
model = LinearRegression()
model.fit(X_train, y_train)
print("Before the data cleaning: ", model.score(X_test, y_test))

###
df = df_train.copy()
corrmat = df.corr()
cols = corrmat.nlargest(5, 'SalePrice') 
df3 = df[cols.index]
X, y = df3.values[:,1:], df3.values[:,0]
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)
model = LinearRegression()
model.fit(X_train, y_train)
print("After the data cleaning: ", model.score(X_test, y_test))

# 모두 사용:
- int64 타입이 아닌 컬럼들은 모두 LabelEncoding() or OneHotEncoding()

In [ ]:
df = df_train.copy()
df.columns

In [ ]:
df.head()

In [ ]:
from sklearn.preprocessing import LabelEncoder, OneHotEncoder
print(type(df.dtypes))    # series
len(df.dtypes)

In [ ]:
df.dtypes

In [ ]:
# string 인 column 은 모두 LabelEncoder
for i in range(1, len(df.dtypes)-1):
    if df.dtypes[i] != 'int64':
        df.iloc[:,i] = LabelEncoder().fit_transform(df.iloc[:,i])

In [ ]:
(df.dtypes != 'int64').sum()

In [ ]:
X, y = df.values[:,1:-1], df.values[:,-1]
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)
model = LinearRegression()
model.fit(X_train, y_train)
print("Using all the featurs: ", model.score(X_test, y_test))

# 연습

### Exercise for skewness, kurtosis

In [ ]:
sample_df = pd.DataFrame(10*np.random.randn(10000,1))
sample_df.head()
print(sample_df.skew(), sample_df.kurtosis())   # excess kurtosis = kurtosis -3
sns.distplot(sample_df, color='b')

- kurtosis(첨도): b=3 정규분포, b>3 뾰족함, b<3 평평함
- skewness(왜도): a=0 정규분포, a>0 좌측치우침, a<0 우측치우침
- excess kurtosis (초과왜도): defined as kurtosis -3
- Note: The only data values (observed or observable) that contribute to kurtosis in 
  any meaningful way are those outside the region of the peak; i.e., the outliers. 
  Therefore, kurtosis measures outliers only; it measures nothing about the "peak".

### Python print 방법 연습

In [ ]:
# 1. 포맷 문자열 (f-문자열)
table = {'Sjoerd': 4127, 'Jack': 4098, 'Dcab': 7678}
# table.items()              # dict.items(), dict.keys(), dict.values()
for name, phone in table.items():
    print(f'{name:7s} ==> {phone:7d}')

In [ ]:
# 2. 문자열 format 메쏘드
print('The story of {1}, {0}, and {other}.'.format('Bill', 'Manfred', other='Georg'))

In [ ]:
# 3. 예전의 문자열 포맷팅 (use %) - sprintf() 스타일
print('%(language)s has %(number)03d quote types.' % {'language': "Python", "number": 2})

### Correlation
- np.corrcoef() and df.corr()
- df.corr() is NaN tolerant whereas np.corrcoef is not.

In [ ]:
a = np.array([1, 2, 3, 4, 5, np.nan])
b = np.array([2, 3, 4, 5.25, np.nan, 100])
print(np.corrcoef(a,b))
# ----
print(np.ma.masked_invalid(a),np.ma.masked_invalid(b))
np.ma.corrcoef(np.ma.masked_invalid(a), np.ma.masked_invalid(b))

In [ ]:
a = np.array([1, 2, 3, 4, 5, np.nan])
b = np.array([2, 3, 4, 5.25, np.nan, 100])
df = pd.concat([pd.DataFrame(a), pd.DataFrame(b)], axis=1)
df.corr()

### exercise 

In [ ]:
array = np.array([[1, np.nan, 3], [4, 5, np.nan]])
print(pd.isna(array))
print(pd.isna(array[0]))

df = pd.DataFrame([['ant', 'bee', 'cat'], ['dog', None, 'fly']],
                 columns=['A','B','C']); df

df['A'] # column name 0 (feature), not a row index (index)

df[['A','B']]

df[:1].values == df.iloc[:1].values

pd.isna(df)